In [15]:
import re
import jieba
import numpy as np

In [16]:
vocab_dict=[]
with open('vocab.txt',encoding='utf-8') as f:
    for i in f:
        i=i.replace('\n','')
        vocab_dict.append(i)

In [17]:
stopwords=[]
with open(r'./stop.txt', 'r', encoding='utf-8') as f:
    for line in f:
        if len(line)>0:
            stopwords.append(line.strip())

In [18]:
class Config():
    max_sequence_length = 500 # 最長序列長度為n個字
    min_word_frequency = 3 # 出現頻率小於n的話 ; 就當成罕見字
    
    vocab_size = None
    category_num = None
    
    choose_model = 'lstm' # 想要使用的模型 ex lstm; rnn; gru
    embedding_dim_size =300 # 詞向量維度
    num_layer = 1 # 層數
    num_units = [128] # 神經元
    learning_rate = 0.0001 # 學習率         
    keep_prob = 0.8 
    
    batch_size = 64 # mini-batch
    epoch_size = 30 # epoch
    
    save_path = 'best_validation' # 模型儲存檔名
    
config = Config()

In [54]:
ans='你收到「領到3000消費券可以直接到全聯社換全聯禮券」、「他多加800塊給你總共3800」、「可到全聯社慢慢買」這樣的謠言訊息嗎？後續這樣的訊息貼文又出現了各種變形版本誤導。事實上關於政府所發行的「振興三倍券」並非消費券，且不能購買禮券，對此全聯福利中心也有在官方網站發出澄清，只是後來在社群媒體與通訊軟體又出現各種變形說法，務必以官方資料為主。'
# ans='又到了四年一度的閏年。在許多歐洲國家，2月29日是女性專屬的求婚日，在這天求婚不只不會被貼上標籤，男生如果拒絕還得受罰！女性選在閏年向男性求婚的習俗從何而來？最常見的說法是源自5世紀的愛爾蘭。據傳聖布麗姬（St. Brigid）向聖派翠克主教（St. Patrick）抱怨，女生等男生求婚要等很久，於是主教就規定在閏年的2月29日，女性可以求婚。後來這項規範傳入蘇格蘭，瑪格麗特皇后（Queen Margaret）據此頒布法令，讓蘇格蘭女性可以在229當天向男性求婚，而且對方如過拒絕還要受罰。男生如果拒絕求婚，就要給女方一個吻或一件絲綢洋裝，或是一雙手套。在部分上層歐洲社會中，還要給到12雙手套，這樣才能讓女生把手遮起來，避免被別人發現沒戴婚戒。至於為什麼會有罰男方獻吻的說法，則是因為有一說是聖派翠克主教一答應讓女生求婚，聖布麗姬立刻下跪向主教求婚，主教拒絕了，但給她一個吻並送她一件絲綢長袍。不過上述故事應該都是杜撰的。聖布麗姬不一定是真實存在的人物，就算真有其人，聖派翠克主教過世的時候，聖布麗姬也還是個未滿10歲的小女孩。瑪格麗特皇后則是7歲就過世了，不太可能真的立過這條法令。女生229才能求婚引發平權爭議不管實際上是怎麼開始的，這套傳統就此傳承了下來。但隨著兩性平權的意識越來越普及，女性選在閏年求婚的說法也遭受了抨擊。熟悉這個傳統的美國學者柏金（Katherine Parkin）就認為，在這個女性地位逐漸提升的年代，特別准許女生每四年可以求婚一次，實在可笑，甚至有羞辱之嫌。但也有人認為，這項傳統的存在其實也是在鼓勵女性拋開傳統枷鎖，當她們所愛的人不敢開口的時候，勇敢站出來主導情勢，從這點來看或許也沒有這麼違反女權的概念。實際上還真的蠻多女生挑在229求婚的。2008年，來自英國的梅特卡夫（Sally Metcalf）就選在那一天向長跑10年的男友求婚成功。她說，「哪一天都好，不過在2月29日訂婚確實讓我覺得我們蠻特別的。」她也鼓勵其他女性不要再等了，直接開口問，只要對方愛你，就不該拒絕。在愛爾蘭和芬蘭，也有很多人相信229是個求婚幸運日，愛爾蘭更有一說是在229求婚可以降低未來的離婚機率。可惜沒有統計數據證明在閏年結婚或訂婚的人比較有可能白頭偕老，在男女平等的時代，誰開口似乎也沒那麼重要。但是女孩，如果妳一直找不到機會或提不起勇氣，不妨就選在這個連假給親愛的他一個驚喜吧！閏年傳統番外篇：有人相信229是幸運日，但也有人認為229這個多餘的日子很邪門。依據希臘傳統，如果在閏年結婚，以後就會離婚。選在閏年離婚的人，這輩子再也沒辦法找到幸福了。參考資料：華爾街日報、Irish Central、HuffPost、timeanddate.com'

In [55]:
ans

'你收到「領到3000消費券可以直接到全聯社換全聯禮券」、「他多加800塊給你總共3800」、「可到全聯社慢慢買」這樣的謠言訊息嗎？後續這樣的訊息貼文又出現了各種變形版本誤導。事實上關於政府所發行的「振興三倍券」並非消費券，且不能購買禮券，對此全聯福利中心也有在官方網站發出澄清，只是後來在社群媒體與通訊軟體又出現各種變形說法，務必以官方資料為主。'

In [46]:
def clean_text(text_string):
    text_string = re.sub(r'[^\u4e00-\u9fa5]+', '', text_string)
    return(text_string)
ans = clean_text(ans)
ans

'又到了四年一度的閏年在許多歐洲國家月日是女性專屬的求婚日在這天求婚不只不會被貼上標籤男生如果拒絕還得受罰女性選在閏年向男性求婚的習俗從何而來最常見的說法是源自世紀的愛爾蘭據傳聖布麗姬向聖派翠克主教抱怨女生等男生求婚要等很久於是主教就規定在閏年的月日女性可以求婚後來這項規範傳入蘇格蘭瑪格麗特皇后據此頒布法令讓蘇格蘭女性可以在當天向男性求婚而且對方如過拒絕還要受罰男生如果拒絕求婚就要給女方一個吻或一件絲綢洋裝或是一雙手套在部分上層歐洲社會中還要給到雙手套這樣才能讓女生把手遮起來避免被別人發現沒戴婚戒至於為什麼會有罰男方獻吻的說法則是因為有一說是聖派翠克主教一答應讓女生求婚聖布麗姬立刻下跪向主教求婚主教拒絕了但給她一個吻並送她一件絲綢長袍不過上述故事應該都是杜撰的聖布麗姬不一定是真實存在的人物就算真有其人聖派翠克主教過世的時候聖布麗姬也還是個未滿歲的小女孩瑪格麗特皇后則是歲就過世了不太可能真的立過這條法令女生才能求婚引發平權爭議不管實際上是怎麼開始的這套傳統就此傳承了下來但隨著兩性平權的意識越來越普及女性選在閏年求婚的說法也遭受了抨擊熟悉這個傳統的美國學者柏金就認為在這個女性地位逐漸提升的年代特別准許女生每四年可以求婚一次實在可笑甚至有羞辱之嫌但也有人認為這項傳統的存在其實也是在鼓勵女性拋開傳統枷鎖當她們所愛的人不敢開口的時候勇敢站出來主導情勢從這點來看或許也沒有這麼違反女權的概念實際上還真的蠻多女生挑在求婚的年來自英國的梅特卡夫就選在那一天向長跑年的男友求婚成功她說哪一天都好不過在月日訂婚確實讓我覺得我們蠻特別的她也鼓勵其他女性不要再等了直接開口問只要對方愛你就不該拒絕在愛爾蘭和芬蘭也有很多人相信是個求婚幸運日愛爾蘭更有一說是在求婚可以降低未來的離婚機率可惜沒有統計數據證明在閏年結婚或訂婚的人比較有可能白頭偕老在男女平等的時代誰開口似乎也沒那麼重要但是女孩如果妳一直找不到機會或提不起勇氣不妨就選在這個連假給親愛的他一個驚喜吧閏年傳統番外篇有人相信是幸運日但也有人認為這個多餘的日子很邪門依據希臘傳統如果在閏年結婚以後就會離婚選在閏年離婚的人這輩子再也沒辦法找到幸福了參考資料華爾街日報'

In [47]:
ans_seg=[]
ans_seg.append([j for j in jieba.cut(ans, cut_all=False) if j not in stopwords])


In [48]:
ans=np.zeros(config.max_sequence_length)
for i in ans_seg:
    for j ,v in enumerate(i) :
        for index ,value in enumerate(vocab_dict):
            if v==value:
                ans[j]=index
ans=ans.reshape(-1,config.max_sequence_length)  

In [49]:
import sklearn.preprocessing
from sklearn.model_selection import train_test_split

 
train_x, train_y = np.load("train_x.npy"),  np.load("train_y.npy")
config.vocab_size = sum(1 for line in open(r"vocab.txt",encoding='utf-8'))
config.category_num = train_y.shape[1]

In [50]:
import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn
    
class TextRNN(object):
    def __init__(self, config):
        self.config = config
        
        # 四個等待輸入的data
        self.batch_size = tf.placeholder(tf.int32, [] , name = 'batch_size')
        self.keep_prob = tf.placeholder(tf.float32, [], name = 'keep_prob')
        
        # Initial
        self.x = tf.placeholder(tf.int32, [None, self.config.max_sequence_length] , name = 'x')
        self.y_label = tf.placeholder(tf.float32, [None, self.config.category_num], name = 'y_label')
        self.choose_model = config.choose_model
        self.rnn()
    # Get LSTM Cell
    def cell(self, num_units):
        #BasicLSTMCell activity => default tanh
        if self.choose_model == "lstm":
            #可以設定peephole等屬性
            LSTM_cell = rnn.LSTMCell(num_units, initializer = tf.random_uniform_initializer(-0.1, 0.1, seed = 2)) 
        elif self.choose_model == "basic":
            #最基礎的，沒有peephole
            LSTM_cell = rnn.BasicLSTMCell(num_units = num_units, forget_bias = 1.0, state_is_tuple = True) 
        else:
            LSTM_cell = rnn.GRUCell(num_units)

        return rnn.DropoutWrapper(LSTM_cell, output_keep_prob = self.keep_prob)
    
    def rnn(self):
        """RNN模型"""
        # 詞向量映射
        with tf.device('/cpu:0'):
            embedding = tf.get_variable('embedding', [self.config.vocab_size, self.config.embedding_dim_size])
            embedding_inputs = tf.nn.embedding_lookup(embedding, self.x)
            self.emb=embedding_inputs
        # RNN Layers
        with tf.name_scope('layers'):
            with tf.name_scope('RNN'):
                LSTM_cells = rnn.MultiRNNCell([self.cell(int(self.config.num_units[_])) for _ in range(self.config.num_layer)])
                # x_shape = tf.reshape(self.x, [-1, self.config.truncate, self.config.vectorSize])
                
            with tf.name_scope('output'):
                init_state = LSTM_cells.zero_state(self.batch_size, dtype = tf.float32)
                outputs, final_state = tf.nn.dynamic_rnn(LSTM_cells, inputs = embedding_inputs, 
                                                        initial_state = init_state, time_major = False, dtype = tf.float32)
                
        # Output Layer
        with tf.name_scope('output_layer'):
            # 全連接層，後面接dropout以及relu激活
            fc1 = tf.layers.dense(outputs[:, -1, :], int(self.config.num_units[len(self.config.num_units)-1]))
            fc1 = tf.contrib.layers.dropout(fc1, self.keep_prob)
            fc1 = tf.nn.relu(fc1)
                
            # 分類器
            y = tf.layers.dense(fc1, self.config.category_num, name = 'y')
            
        self.y_pred_cls = tf.argmax(y, axis = 1) #預測類別
        with tf.name_scope('cross_entropy'):
            with tf.name_scope('total'):
                self.softmax = tf.nn.softmax_cross_entropy_with_logits(labels = self.y_label, logits = y)
                self.cross_entropy = tf.reduce_mean(self.softmax)
                self.y=tf.nn.softmax(y)  
                self.cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = self.y_label, logits = y))
                
        with tf.name_scope('train'):
            self.train_step = tf.train.AdamOptimizer(self.config.learning_rate).minimize(self.cross_entropy)

        with tf.name_scope('accuracy'):
            with tf.name_scope('correct_prediction'):
                self.correction_prediction = tf.equal(self.y_pred_cls, tf.argmax(self.y_label, axis = 1))
            with tf.name_scope('accuracy'):
                self.accuracy = tf.reduce_mean(tf.cast(self.correction_prediction, tf.float32))

In [53]:

def feedData1(x_batch, keep_prob, batch_size, model):
    feed_dict1 = {
        model.x: x_batch,
#         model.y_label: y_batch,
        model.keep_prob: keep_prob,
        model.batch_size: batch_size
    }
    return feed_dict1
tf.reset_default_graph()
model = TextRNN(config)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver()
    saver.restore(sess = sess, save_path = config.save_path)  # 讀取保存的模型
    test_predict_label,y  = sess.run([ model.y_pred_cls,model.y], feed_dict = feedData1(ans, 1.0 ,len(ans), model))    
    c=float(np.max(y))
    if test_predict_label==0:
        print('有%.2f的機率是假新聞'%(float(np.max(y))))
    else:
        print('有%.2f的機率是真新聞唷'%(float(np.max(y))))

INFO:tensorflow:Restoring parameters from best_validation
有0.83的機率是真新聞唷
